[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/safevideo/autollm/blob/quickstart-collab/examples/quickstart.ipynb)

## 0. Preparation

- Install latest version of autollm:

In [1]:
!pip install autollm -Uq
!pip install nest-asyncio -Uq

- Import required modules:

In [2]:
# import required functions, classes
from autollm import AutoQueryEngine, AutoFastAPI
from autollm.utils.document_reading import read_files_as_documents
import uvicorn
import os
import nest_asyncio
nest_asyncio.apply()

- Set your OpenAI API key in order to use default gpt-3.5-turbo model:

In [ ]:
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

## 1. Read Files as Documents

- Either set input directory or input files:

In [4]:
# input_dir = 'data/llama_index/documents'
input_files = ["../README.md"]
recursive = True    # Whether to recursively search in subdirectories

documents = read_files_as_documents(input_files=input_files, recursive=recursive)

INFO:autollm.utils.document_reading:Found 1 'documents'.


## 2. Configuration of AutoQueryEngine

### Basic Usage

- You can completely skip configuration if you want to use default settings.

- 🌟 **pro tip**: autollm defaults to lancedb as the vector store since it is lightweight, scales from development to production and is 100x cheaper than alternatives!

In [5]:
query_engine = AutoQueryEngine.from_parameters(documents=documents)

/home/seekndestroy/mambaforge/envs/aidocs/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating embeddings: 100%|██████████| 4/4 [00:00<00:00,  6.19it/s]


### Advanced Usage

- You can configure the AutoQueryEngine to your needs:

In [ ]:
system_prompt = "You are an friendly ai assistant that help users find the most relevant and accurate answers to their questions based on the documents you have access to. When answering the questions, mostly rely on the info in documents."

query_wrapper_prompt = '''
The document information is below.
---------------------
{context_str}
---------------------
Using the document information and mostly relying on it,
answer the query.
Query: {query_str}
Answer:
'''

enable_cost_calculator = True

# llm params
model = "gpt-3.5-turbo"

# vector store params
vector_store_type = "LanceDBVectorStore"
# specific params for LanceDBVectorStore
uri = "tmp/lancedb" 
table_name = "vectors"

# service context params
chunk_size = 1024

# query engine params
similarity_top_k = 5

In [ ]:
llm_params = {"model": model}
vector_store_params = {"vector_store_type": vector_store_type, "uri": uri, "table_name": table_name}
service_context_params = {"chunk_size": chunk_size}
query_engine_params = {"similarity_top_k": similarity_top_k}

query_engine = AutoQueryEngine.from_parameters(documents=documents, system_prompt=system_prompt, query_wrapper_prompt=query_wrapper_prompt, enable_cost_calculator=enable_cost_calculator, llm_params=llm_params, vector_store_params=vector_store_params, service_context_params=service_context_params, query_engine_params=query_engine_params)

## 3. Ask Anything to Your Documents

In [7]:
response = query_engine.query("who are you")

LLM Prompt Token Usage: 743
LLM Completion Token Usage: 22
LLM Total Token Cost: $0.001159


In [9]:
response.response

'I am an expert Q&A system designed to help users find relevant and accurate answers to their questions.'